In [2]:
using AdaGram

In [4]:
vm, dict = load_model("../models/adagram/v1_d100.adagram_model")

In [ ]:
word = "apple"
prior_probs = expected_pi(vm, dict.word2id[word])
for ii in 1:5
    println(prior_probs[ii],"\t",nearest_neighbors(vm, dict, word, ii, 10))
    println()
end